In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import joblib 
import numpy as np
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.svm import SVC 

In [2]:
# Function to build the autoencoder


In [3]:

def define_encoder ( 
    X_train, X_test, 
    encoding_dim =16,
    dense_shape = ( 128, 64 ), 
    activation ='relu', 
    dec_activation ='sigmoid', 
    loss= 'binary_crossentropy', 
    learning_rate = 0.001, 
    epochs= 100, 
    batch_size = 75 , 
    validation_split = .3,
    verbose = 1 
    ): 

    # # Deep Autoencoder Configuration
    input_dim = X_train.shape[1] # Number of features (55 in your case)
    # encoding_dim = 16  # Dimension of encoded data. Adjust as needed 10 12 14 
    
    # Encoder
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(dense_shape[0], activation=activation)(input_layer) 
    encoder = Dense(dense_shape[1], activation=activation)(encoder)
    
    encoder = Dense(encoding_dim, activation=activation)(encoder)
    
    # Decoder
    decoder = Dense(dense_shape[1], activation=activation)(encoder)
    decoder = Dense(dense_shape[0], activation=activation)(decoder)
    decoder = Dense(input_dim, activation=dec_activation)(decoder)
    
    # Autoencoder Model
    autoencoder = Model(input_layer, decoder)
    autoencoder.compile(optimizer=Adam(learning_rate=learning_rate),
                        loss=loss)#bce
    
    # Train the autoencoder
    autoencoder.fit(X_train, X_train, epochs=epochs,
                    batch_size=batch_size, 
                    validation_split=validation_split,
                    verbose=verbose
                    )
    
    # Extract the encoder model
    encoder_model = Model(input_layer, encoder)
    
    # Reduce dimensionality of the data
    X_train_encoded = encoder_model.predict(X_train)
    
    X_test_encoded = encoder_model.predict(X_test)
    
    return X_train_encoded, X_test_encoded

def encode_multiple(
    X_train , X_test,  y_train =None, y_test =None, # 
    encoding_dims =[10], #  15, 20, 25, 30, 35, 40, 45, 50, 55], 
    dense_shapes = [(512, 256), ( 128, 64 ), (32, 16), (8, 4)], 
    activations =[ 'relu', 'selu', 'elu',  'tanh'], 
    dec_activations =[ 'sigmoid', 'softmax'], 
    loss= 'binary_crossentropy', 
    learning_rates = [0.001,0.01, 0.1, 1] , 
    epochs= 100, 
    batch_size = 75 , 
    validation_split = .3,
    verbose = 1,  
    savejob=True, 
    savefile ='dim_autoenc_data_10.joblib'
    
    ): 
    ratenames =['lr0001', 'lr001', 'lr01', 'lr1']
    save_dict ={}
    for encoding_dim in encoding_dims: 
        for dense_shape  in dense_shapes:
            if verbose: 
                print("{:*^100}".format(" {dense_shape} "))
            for activation in activations : 
                for dec_activation in dec_activations: 
                    for lrname, learning_rate in zip ( ratenames, learning_rates) :
                        Xtrain_enc, Xtest_enc= define_encoder(
                            X_train = X_train, X_test = X_test,
                            encoding_dim= encoding_dim, 
                            dense_shape = dense_shape, 
                            activation= activation, 
                            dec_activation= dec_activation, 
                            learning_rate= learning_rate, 
                            loss=loss, 
                            epochs= epochs, 
                            batch_size= batch_size, 
                            validation_split= validation_split, 
                            verbose= verbose, 
                    )  # dim10.512_256.relu.sigmoid.lr0001 [train]
                        key = f"dim{encoding_dim}.{dense_shape[0]}_{dense_shape[1]}.{activation}.{dec_activation}.{lrname}"
                        save_dict[ key] = {"train":( Xtrain_enc, y_train), 
                                           "test": (Xtest_enc, y_test)
                                           } 
                    if verbose: 
                        print( f" ----> Running  key = {key}")
    if savejob: 
        joblib.dump ( save_dict, filename = savefile  ) 
        
    return save_dict 

In [4]:
# Split the dataset 
seed =42 
DATA= joblib.load ("DATA.joblib")
def type_data ( dict_data , kind ='StandardScaler'): 
    X, y = dict_data [kind ] 
    
    return X, y 
# X_train, X_test, ytrain, y_test = train_test_split(
#     X, y, test_size=0.3, random_state=seed)

def save_encode_data ( dict_data , kinds = [ "StandardScaler", "RobustScaler", "MinMaxScaler"], 
                      savejob =True, 
                      savefile ='globalEncodeDATA.joblib', 
                      **kws
                      ): 
    
    global_data_dict ={}
    for kind in  kinds: 
        X, y = type_data ( dict_data, kind )
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=seed)
        
        enc_dict = encode_multiple ( X_train = X_train, 
                          X_test = X_test, 
                          y_train =y_train, 
                          y_test =y_test, 
                          **kws
                          )
  
        # [ standardSCaler]= dim10.512_256.relu.sigmoid [train] => X_train_enc, y_train
        # [ standardSCaler]= dim10.512_256.relu.sigmoid [test] => X_test_enc, y_test 
        global_data_dict [kind] = enc_dict
        
    if savejob: 
        joblib.dump ( global_data_dict, filename =savefile )
        
    return  global_data_dict


In [5]:
# get separate data for dim =10 et dim =25 

filename = "globalEncodeDATA.joblib" 
global_data = joblib.load (filename )

# get data standarscaler data 
data_sc = global_data.get("StandardScaler")
keys_data_sc = data_sc.keys() 
key_dim10_data ={} ; key_dim25_data={}
for key, value in  data_sc.items() : 
    if str(key).find("dim10") >=0: 
        key_dim10_data[key] =value 
    else: key_dim25_data[key] =value 

In [6]:
# %% 
# # SET  best estimator 
# # XXX IMPORTANT (1)
# %% 
from sklearn.metrics import    accuracy_score, precision_score, recall_score,f1_score 
from watex.utils.validator import get_estimator_name  
from watex.utils.box import Boxspace 
from sklearn.preprocessing import StandardScaler 

# PREDICT 
def auto_enc_predict ( dict_data , best_estimator, verbose =False ) : 
    # stored dim_evaluate_results '
    boxg= {}
    dim_results ={}
    for ii, key in enumerate (dict_data.keys()) : 
        if verbose: 
            print("{:=^200}".format( f" ---{ii:02}---> Running {get_estimator_name(best_estimator).upper()}  with key = {key}")) 
            
        X_train_enc, y_train = dict_data[key]['train']
        X_test_enc, y_test = dict_data [key]['test']
        
        if np.isnan ( X_train_enc).any() : 
            continue 
        if get_estimator_name(best_estimator) in ("SVM", "SVC"): 
            scaler = StandardScaler () 
            X_train_enc = scaler.fit_transform ( X_train_enc )
            X_test_enc = scaler.fit_transform (X_test_enc )
        best_estimator.fit( X_train_enc, y_train )
        y_pred = best_estimator.predict(X_test_enc)
        ac_score = accuracy_score(y_test, y_pred)
        rec_score = recall_score(y_test, y_pred)
        prec_score = precision_score(y_test, y_pred)
        f1_scor = f1_score(y_test, y_pred)
        component =str(key)[:5]
        if verbose:
            
            print("-"*110)
            print("{:^110}".format(get_estimator_name(best_estimator)+(
                  "''" if not component else f"({str(component)})"))
                  )
            print("-"*110)
            print("|{:^26}|{:^26}|{:^26}|{:^26}|".format(
                "Accuracy", "F1_score", "Precision", "Recall")
            )
            print("-"*110)
            print("|{:>26}|{:>26}|{:>26}|{:>26}|".format(
                *[round(sc, 4) for sc in (
                    ac_score, f1_scor, prec_score, rec_score)])
                  )
            print("-"*110)

        dict_scores = dict(estimator_= get_estimator_name(best_estimator), 
                           component_=component,
                           accuracy_=ac_score,
                           f1_score_=f1_scor,
                           precision_=prec_score,
                           recall_=rec_score
                           )
        scores = [ ac_score, f1_scor, prec_score,rec_score ]
        
        dim_results[key] = ( scores , dict_scores) 
        
        boxg [key] = Boxspace(**dict_scores)
        
    return dim_results, boxg 

In [7]:
#%% 

# run predict 
# XXXX IMPORTANT (2)

In [8]:
# %% 
def get_key_best_score ( dim_data , verbose = True , best_estimator=None,
                        best_params=None, many=None ): 
    
    # construct best key 
    key_best ={}
    key_vector = list(dim_data.keys ())
    key_values = dim_data.values () 
    
    score_val = [ val for val , _ in key_values ]
     
    score_ar = np.vstack ( score_val ) 
    # print(score_ar.shape)
    # from acccuracy ( index =0 ) , get_argmax 
    best_index  = np.argmax ( score_ar [:, 0]) 
    # find the key from index 
    key_best[ key_vector [best_index] ] = score_ar [best_index, :]
    
    if verbose: 
        if not many: 
            print("+"*320)
            print("{:^40} | {:^15} | {:^150} | {:^100}".format(
                "DATA", "Estimators", "best_params",  "Test scores (acc): acc-std"))
            print("+"*320)
        for key, value in key_best.items(): 
            if best_estimator is None: best_estimator ='', 
            if best_params is None: best_params=''
            print("{:^40} | {:^15} | {:^150} | {:^100}".format(
                key, best_estimator,  str(best_params),  
                "acc:{:<10} - f1_score: {:<10} - precision:{:<10} - recall {:<10}".format(
                    round ( value[0], 4) , round(value[1], 4), 
                    round(value[2], 4) ,round( value[3], 4) ) 
                )
            )
        if not many: 
            print("+"*320)
    
    return key_best 

In [9]:
# %% 
# IMPORTANT (3) 
#%% 

In [10]:
# =================================DIMENSION 10===============================
# Refinement 
# rewrite the code to work with all 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from  sklearn.naive_bayes import GaussianNB
from watex.exlib.gbm import XGBClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.svm import SVC 
from lightgbm import LGBMClassifier
from watex.utils.validator import get_estimator_name 

In [11]:
# %% 

# globdict_10= save_encode_data (DATA,savejob =True, encoding_dims=[10], 
#                             savefile= "Auto_encode_dim10_only.joblib" )
# get separate data for dim =10 et dim =25 
# %%
filename_10 = "Auto_encode_dim10_only.joblib" 
global_data_10 = joblib.load (filename )
 
# get data standarscaler data 
data_sc_10 = global_data_10.get("StandardScaler")
keys_data_sc_10 = data_sc_10.keys() 
key_dim10_data_new ={} 
for key, value in  data_sc.items() : 
    if str(key).find("dim10") >=0: 
        key_dim10_data_new[key] =value 
 

# get the key best data to take all the best estimators 

keybest_10 = joblib.load ("keybest_10.joblib")["keybest_datacomp10"]

map_estimators = {
    "LR":LogisticRegression, 
    'KNN':KNeighborsClassifier,
    'DT':DecisionTreeClassifier,
    "RF": RandomForestClassifier, 
    "ADA":AdaBoostClassifier, 
    "XGB":XGBClassifier, 
    "NB":GaussianNB, 
    "LGBM":LGBMClassifier, 
    "LDA":LinearDiscriminantAnalysis,
    "SVM":SVC, 
    # "Ridge": "Ridge"
}

best_estimators = [] ; best_params_10=[]
for _, est_acronym, search_name, cv_val, best_param, *_ in  keybest_10: 
    for acr in map_estimators.keys(): 
        if str(acr).find(est_acronym)>=0: 
            best_estimators.append ( map_estimators[acr](**dict (best_param )) )
            best_params_10.append (best_param)

# run predict

C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
# XXXX IMPORTANT (2)
dim_results_10 , boxg_10 = [], [] 
for best_estimator in best_estimators: 
    try:
        r, b = auto_enc_predict( 
            # key_dim25_data, 
            key_dim10_data_new, 
            best_estimator ,
            verbose =True, 
            )
        dim_results_10.append (r) 
        boxg_10.append( b)
    except: 
        dim_results_10.append (r) 
        boxg_10.append( b)
        
        continue 
    
# save this into a dict 
save_dict_10 = {"dim_results_10": dim_results_10,  
                'boxg_10':boxg_10}
joblib.dump ( save_dict_10 , "save_dict_10.joblib")

========================================================== ---00---> Running LOGISTICREGRESSION  with key = dim10.512_256.relu.sigmoid.lr0001===========================================================


C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


--------------------------------------------------------------------------------------------------------------
                                          LogisticRegression(dim10)                                           
--------------------------------------------------------------------------------------------------------------
|         Accuracy         |         F1_score         |        Precision         |          Recall          |
--------------------------------------------------------------------------------------------------------------
|                    0.7912|                     0.869|                    0.7975|                    0.9545|
--------------------------------------------------------------------------------------------------------------
=========================================================== ---01---> Running LOGISTICREGRESSION  with key = dim10.512_256.relu.sigmoid.lr001===========================================================
------------------------

C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


--------------------------------------------------------------------------------------------------------------
                                          LogisticRegression(dim10)                                           
--------------------------------------------------------------------------------------------------------------
|         Accuracy         |         F1_score         |        Precision         |          Recall          |
--------------------------------------------------------------------------------------------------------------
|                    0.8022|                     0.875|                    0.8077|                    0.9545|
--------------------------------------------------------------------------------------------------------------
=========================================================== ---05---> Running LOGISTICREGRESSION  with key = dim10.512_256.relu.softmax.lr001===========================================================
------------------------

...

C:\Users\BZ\.conda\envs\DanP\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\xgboost\sklearn.py:1224: UserWarning:

--------------------------------------------------------------------------------------------------------------
                                             XGBClassifier(dim10)                                             
--------------------------------------------------------------------------------------------------------------
|         Accuracy         |         F1_score         |        Precision         |          Recall          |
--------------------------------------------------------------------------------------------------------------
|                    0.7912|                     0.869|                    0.7975|                    0.9545|
--------------------------------------------------------------------------------------------------------------
============================================================== ---00---> Running GAUSSIANNB  with key = dim10.512_256.relu.sigmoid.lr0001===============================================================
------------------------

C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\naive_bayes.py:516: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
C:\Users\BZ\.conda\envs\DanP\lib\site-packages\sklearn\naive_bayes.py:516: RuntimeWarning: invalid value encountered in div

--------------------------------------------------------------------------------------------------------------
                                              GaussianNB(dim10)                                               
--------------------------------------------------------------------------------------------------------------
|         Accuracy         |         F1_score         |        Precision         |          Recall          |
--------------------------------------------------------------------------------------------------------------
|                    0.2747|                       0.0|                       0.0|                       0.0|
--------------------------------------------------------------------------------------------------------------
=============================================================== ---24---> Running GAUSSIANNB  with key = dim10.128_64.relu.sigmoid.lr0001===============================================================
------------------------

['save_dict_10.joblib']

In [14]:
# !!!! %% THIS PART IS VERY IMPORTANT, IT SHOWS THE BEST SCORE FOR EACH CLASSIFIER ON DIMENSION 10, USE HERE ONLY

for ii, (enco_dim_r, best_estimator) in  enumerate ( zip ( dim_results_10, best_estimators )): 
    # print("ESTIMATOR=...", best_estimator)Recall          |
#------------------------------------------------------------------------------------------------
    auto_enc_results = get_key_best_score ( 
        dim_data= enco_dim_r, 
        best_estimator= get_estimator_name ( best_estimator), 
        best_params= best_params_10[ii], 
        verbose=True, 
        many= None if ii==0 or ii== (len(dim_results_10) -1) else True 
        )

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                  DATA                   |   Estimators    |                                                                      best_params                                                                       |                                      Test scores (acc): acc-std                                     
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    dim10.512_256.elu.softmax.lr0001     | 

In [ ]:
# %% EXECUTE THIS FOLLOWING CODE IF YOU WANT TO SEE DETAILS OF SVM ONLY
# predict with single SVM estimator 
best_estimator = SVC (kernel="linear", gamma ='auto', degree =1,coef0=9, C=2.0  )
    
keys_10_predict = [ 
    "dim10.128_64.elu.softmax.lr0001", 
    "dim10.512_256.relu.softmax.lr01",  
    "dim10.512_256.elu.softmax.lr0001",
    "dim10.8_4.relu.softmax.lr001" ,
    "dim10.128_64.elu.softmax.lr0001" ,
    "dim10.8_4.relu.softmax.lr001", 
    "dim10.512_256.relu.sigmoid.lr0001" ,
    "dim10.32_16.elu.softmax.lr001" ,
]

key_10_data= { key: key_dim10_data_new [key] for key  in keys_10_predict}

In [ ]:
# %% 
# use the key_10_data for svm prediction 

    
enco_dim_results , boxo = auto_enc_predict( 
    key_10_data, 
    best_estimator,
    verbose =True, 
    ) 